<center> <h3> Final Project | ANLY-580 </h3>
<h4>  Georgetown University | Fall 2018</h4>
<center>

#### Prep

In [3]:
# set working directory
import os
import sys
project_root = '/Users/kgedney/Documents/georgetown/anly580/anly580-wine-project'
os.chdir(project_root)

In [130]:
# install packages
import numpy as np
import pandas as pd

In [14]:
# load data
df = pd.read_csv('data/cellarinsider_raw.csv', encoding='utf-8')

#### Data Preprocessing

In [16]:
# check how many samples per class
df.describe()

,Unnamed: 0,note_id,rating,year
count,17713.000000,17713.000000,17713.000000,17713.000000
mean,8856.000000,0.952408,91.805058,2002.216225
std,5113.446995,2.125400,4.609024,35.300473
min,0.000000,0.000000,0.000000,1001.000000
25%,4428.000000,0.000000,89.000000,1999.000000
50%,8856.000000,0.000000,92.000000,2009.000000
75%,13284.000000,1.000000,95.000000,2013.000000
max,17712.000000,23.000000,100.000000,2017.000000


In [29]:
df[df.region == 'California']

df['is_pinot_noir'] = df.bottle.apply(lambda x: 'pinot noir' in x.lower())
df['is_cabernet_sauvignon'] = df.bottle.apply(lambda x: 'cabernet sauvignon' in x.lower())

In [30]:
print(df.is_pinot_noir.sum())
print(df.is_cabernet_sauvignon.sum())

49
933


In [ ]:
# notes:
# maybe use kaggle data plus bordeaux and rhone that I scraped
# train on kaggle then test on my dataset!

#### Text Cleaning

In [ ]:
# make lowercase, strip diacritics, strip punctuation

In [ ]:
# drop leakage words - ie. california, bordeaux, granache, etc.

### Bordeaux vs. Not Bordeaux

#### Prep

In [134]:
# install packages
import numpy as np

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import tensorflow as tf
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from keras.models import Model, Sequential

from keras.layers import Input, Dense
from keras.layers import LSTM, Embedding
from keras.layers.core import Dense, Dropout

/Users/kgedney/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [129]:
# assign classes
df['bordeaux_class'] = (df.region == 'Bordeaux').astype(int)

In [147]:
x = df.note.values
y = df.bordeaux_class.values

In [136]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=522)

##### Baseline Model - Linear SVM

In [62]:
# data preprocess
tfidf_vec   = TfidfVectorizer()
x_train_vec = tfidf_vec.fit_transform(x_train)
x_test_vec  = tfidf_vec.transform(x_test)
x_train_vec.shape

/Users/kgedney/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(14170, 8462)

In [104]:
from sklearn.naive_bayes import MultinomialNB
clf             = MultinomialNB().fit(x_train_vec, y_train)
predicted       = clf.predict(x_test_vec)
predicted_score = clf.predict_proba(x_test_vec)[:,1]

print(pd.crosstab(y_test == 1, predicted, colnames=['predicted'], margins=True))
print('---')
print('accuracy', metrics.accuracy_score(y_test, predicted))
print('auc score', metrics.roc_auc_score(y_test, predicted_score))

predicted    0     1   All
row_0                     
False      424   604  1028
True         9  2506  2515
All        433  3110  3543
---
accuracy 0.8269827829523003
auc score 0.9093924778179174


In [109]:
from sklearn.svm import LinearSVC
clf             = LinearSVC().fit(x_train_vec, y_train)
predicted       = clf.predict(x_test_vec)
predicted_score = clf.decision_function(x_test_vec)

print(pd.crosstab(y_test == 1, predicted, colnames=['predicted'], margins=True))
print('---')
print('accuracy', metrics.accuracy_score(y_test, predicted))
print('auc score', metrics.roc_auc_score(y_test, predicted_score))

predicted    0     1   All
row_0                     
False      750   278  1028
True       151  2364  2515
All        901  2642  3543
---
accuracy 0.8789161727349704
auc score 0.9264036404143234


In [111]:
clf.coef_.shape

(1, 8462)

In [131]:
coefs = clf.coef_.squeeze()

vocab = list(tfidf_vec.vocabulary_.items())
vocab = sorted(vocab, key=lambda x: x[1])
vocab = np.array(vocab)

print('low bordeaux words')
print(vocab[np.argsort(coefs)[:5]])
print('--')
print('high bordeaux words')
print(vocab[np.argsort(coefs)[-5:]])

low bordeaux words
[['california' '1308']
 ['grenache' '3541']
 ['garrigue' '3359']
 ['pepper' '5492']
 ['kirsch' '4159']]
--
high bordeaux words
[['blanc' '989']
 ['cedar' '1427']
 ['pomerol' '5711']
 ['pts' '5936']
 ['merlot' '4783']]


##### Model - LSTM

In [148]:
# data preprocess -> create sequences
max_features = 20000
tokenizer    = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(x)
x_sequences  = tokenizer.texts_to_sequences(x)

In [149]:
# pad each sequence to be max length of 256
maxlen = max(len(x) for x in x_sequences)
print(maxlen)
x_sequences = sequence.pad_sequences(x_sequences, maxlen)

256


In [150]:
# convert into matrix with 17713 rows, 256 cols and values 1-9523
x_sequences = np.asmatrix(x_sequences)
x_sequences.shape

(17713, 256)

In [152]:
# split test and train
x_train, x_test, y_train, y_test = train_test_split(x_sequences, y, test_size=0.20, random_state=522)

In [161]:
model = Sequential()

# add embedding layer
model.add(Embedding(input_dim=max_features,
                    output_dim=128))
# add rnn layer
model.add(LSTM(128))
model.add(Dropout(0.2))

# output layer, binary classification
model.add(Dense(1, activation="sigmoid"))

In [162]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])

In [163]:
model.fit(x_train, y_train,
            batch_size=128,
            epochs=5,
            validation_data=(x_test, y_test))

Train on 14170 samples, validate on 3543 samples
Epoch 1/5
14170/14170 [==============================] - 62s 4ms/step - loss: 0.4758 - acc: 0.7783 - val_loss: 0.3372 - val_acc: 0.8637
Epoch 2/5
14170/14170 [==============================] - 69s 5ms/step - loss: 0.2917 - acc: 0.8817 - val_loss: 0.3246 - val_acc: 0.8580
Epoch 3/5
14170/14170 [==============================] - 79s 6ms/step - loss: 0.2510 - acc: 0.8984 - val_loss: 0.2994 - val_acc: 0.8792
Epoch 4/5
14170/14170 [==============================] - 92s 6ms/step - loss: 0.2268 - acc: 0.9117 - val_loss: 0.3334 - val_acc: 0.8631
Epoch 5/5
14170/14170 [==============================] - 90s 6ms/step - loss: 0.2056 - acc: 0.9174 - val_loss: 0.3115 - val_acc: 0.8648


In [168]:
print('accuracy', model.evaluate(x_test, y_test)[1])

3543/3543 [==============================] - 5s 1ms/step
accuracy 0.8648038386894827


In [166]:
# check confusion matrix
y_test_preds       = model.predict(x_test, batch_size=64)
y_test_preds_class = y_test_preds > 0.5

pd.crosstab(y_test, y_test_preds_class[:,0], colnames=['predicted'], margins=True)

predicted,False,True,All
row_0,,,
0,825,203,1028
1,276,2239,2515
All,1101,2442,3543


##### Word2Vec and Fast Text

In [ ]:
# train word embeddings # https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c

##### Word2Vec and T-SNE